In [ ]:
# Import dependencies
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import load_model
import warnings
warnings.filterwarnings("ignore")

In [3]:
# Read in CSV file - convert to DataFrame
full_flight_data = pd.read_csv("../data/full_data_flightdelay.csv")
full_flight_data.head()

,MONTH,DAY_OF_WEEK,DEP_DEL15,DEP_TIME_BLK,DISTANCE_GROUP,SEGMENT_NUMBER,CONCURRENT_FLIGHTS,NUMBER_OF_SEATS,CARRIER_NAME,AIRPORT_FLIGHTS_MONTH,...,PLANE_AGE,DEPARTING_AIRPORT,LATITUDE,LONGITUDE,PREVIOUS_AIRPORT,PRCP,SNOW,SNWD,TMAX,AWND
0,1,7,0,0800-0859,2,1,25,143,Southwest Airlines Co.,13056,...,8,McCarran International,36.08,-115.152,NONE,0.0,0.0,0.0,65.0,2.91
1,1,7,0,0700-0759,7,1,29,191,Delta Air Lines Inc.,13056,...,3,McCarran International,36.08,-115.152,NONE,0.0,0.0,0.0,65.0,2.91
2,1,7,0,0600-0659,7,1,27,199,Delta Air Lines Inc.,13056,...,18,McCarran International,36.08,-115.152,NONE,0.0,0.0,0.0,65.0,2.91
3,1,7,0,0600-0659,9,1,27,180,Delta Air Lines Inc.,13056,...,2,McCarran International,36.08,-115.152,NONE,0.0,0.0,0.0,65.0,2.91
4,1,7,0,0001-0559,7,1,10,182,Spirit Air Lines,13056,...,1,McCarran International,36.08,-115.152,NONE,0.0,0.0,0.0,65.0,2.91


In [7]:
# Inspect the data in our DataFrame
print(full_flight_data.info())
print(full_flight_data.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6489062 entries, 0 to 6489061
Data columns (total 26 columns):
 #   Column                         Dtype  
---  ------                         -----  
 0   MONTH                          int64  
 1   DAY_OF_WEEK                    int64  
 2   DEP_DEL15                      int64  
 3   DEP_TIME_BLK                   object 
 4   DISTANCE_GROUP                 int64  
 5   SEGMENT_NUMBER                 int64  
 6   CONCURRENT_FLIGHTS             int64  
 7   NUMBER_OF_SEATS                int64  
 8   CARRIER_NAME                   object 
 9   AIRPORT_FLIGHTS_MONTH          int64  
 10  AIRLINE_FLIGHTS_MONTH          int64  
 11  AIRLINE_AIRPORT_FLIGHTS_MONTH  int64  
 12  AVG_MONTHLY_PASS_AIRPORT       int64  
 13  AVG_MONTHLY_PASS_AIRLINE       int64  
 14  FLT_ATTENDANTS_PER_PASS        float64
 15  GROUND_SERV_PER_PASS           float64
 16  PLANE_AGE                      int64  
 17  DEPARTING_AIRPORT              object 
 18  LA

### Important Columns
**`DEP_DEL15`:** returns binary response of `0` for NO flight delay (under 15 min) or `1` for a flight delay +15 min

**`DEP_TIME_BLK`:** distance to be flown by the departing aircraft (time block interval)

**`DISTANCE_GROUP`:** distance to be flown by the departing aircraft

**`SEGMENT_NUMBER`:** segment that tail number is on for the day

**`CONCURRENT_FLIGHTS`:** number of flights departing airport within same departure time block (tarmac traffic may cause delays)

**`CARRIER_NAME`:** airline/carrier for departing flight

**`PLANE_AGE`:** age of departing aircraft

**`DEPARTING_AIRPORT`:** departing airport for flight

**`PRCP`:** inches of precipitation for the day

**`SNOW`:** inches of snowfall for the day

**`SNWD`:** inches of snow on ground for the day

**`TMAX`:** max temperature for the day

**`AWND`:** max wind speed for the day


In [9]:
# Drop unnecessary columns for final DataFrame
flight_delay_df = full_flight_data.drop(columns=['NUMBER_OF_SEATS'])
flight_delay_df.head()

,MONTH,DAY_OF_WEEK,DEP_DEL15,DEP_TIME_BLK,DISTANCE_GROUP,SEGMENT_NUMBER,CONCURRENT_FLIGHTS,CARRIER_NAME,AIRPORT_FLIGHTS_MONTH,AIRLINE_FLIGHTS_MONTH,...,PLANE_AGE,DEPARTING_AIRPORT,LATITUDE,LONGITUDE,PREVIOUS_AIRPORT,PRCP,SNOW,SNWD,TMAX,AWND
0,1,7,0,0800-0859,2,1,25,Southwest Airlines Co.,13056,107363,...,8,McCarran International,36.08,-115.152,NONE,0.0,0.0,0.0,65.0,2.91
1,1,7,0,0700-0759,7,1,29,Delta Air Lines Inc.,13056,73508,...,3,McCarran International,36.08,-115.152,NONE,0.0,0.0,0.0,65.0,2.91
2,1,7,0,0600-0659,7,1,27,Delta Air Lines Inc.,13056,73508,...,18,McCarran International,36.08,-115.152,NONE,0.0,0.0,0.0,65.0,2.91
3,1,7,0,0600-0659,9,1,27,Delta Air Lines Inc.,13056,73508,...,2,McCarran International,36.08,-115.152,NONE,0.0,0.0,0.0,65.0,2.91
4,1,7,0,0001-0559,7,1,10,Spirit Air Lines,13056,15023,...,1,McCarran International,36.08,-115.152,NONE,0.0,0.0,0.0,65.0,2.91


## Feature Engineering

In [10]:
# Create feature for weekends and holiday months (peak travel times)
flight_delay_df['IS_WEEKEND'] = flight_delay_df['DAY_OF_WEEK'].isin([6, 7]).astype(int)
flight_delay_df['IS_HOLIDAY'] = flight_delay_df['MONTH'].isin([7, 12]).astype(int)